# 可視化のための情報を記録（summary）

Tensor ---(Summary operation)---> Protocol buffer ---(Summary writer)---> Disk

## Summary operation

- tf.summary.scalar
- tf.summary.image
- tf.summary.audio
- tf.summary.histogram
- tf.summary.tensor (under development)

## Summary writer (tf.summary.FileWriter)

- writer = tf.summary.FileWriter(LOGDIR)
- writer.add_graph(sess.graph)
- writer.add_summary(s, global_step=i)

# TensorBoardで学習を可視化

- [TensorBoard: Visualizing Learning](https://www.tensorflow.org/get_started/summaries_and_tensorboard)
- [Dandelion Mané on GitHub](https://github.com/dandelionmane)
- https://github.com/mamcgrath/TensorBoard-TF-Dev-Summit-Tutorial

- MNIST
- C - P - C - P - F - F

In [1]:
import os, shutil

In [2]:
import tensorflow as tf

In [3]:
LOGDIR = '/tmp/mnist_demo/3'

In [4]:
if os.path.exists(LOGDIR):
    shutil.rmtree(LOGDIR)

## データの読み込み

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## レイヤーを定義

In [6]:
# 畳み込み層
def conv_layer(input, channels_in, channels_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([5, 5, channels_in, channels_out]), name="W")
        b = tf.Variable(tf.zeros([channels_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        act = tf.nn.relu(conv + b)
        pool = tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")        
        tf.summary.histogram("weights", w)
        tf.summary.histogram("bias", b)
        tf.summary.histogram("activations", act)
        return pool

In [7]:
# 全結合層　
def fc_layer(input, channels_in, channels_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([channels_in, channels_out]), name="W")
        b = tf.Variable(tf.zeros([channels_out]), name="B")
        logits = tf.matmul(input, w)+ b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)        
        return logits

## モデルの構築

In [8]:
# 入力と出力を保持するプレースホルダを用意し、入力を変形
x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")
x_image = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image("input", x_image, 3)

# ネットワークを構築
conv1 = conv_layer(x_image, 1, 32, name="conv1")
conv2 = conv_layer(conv1, 32, 64, name="conv2")
flattened = tf.reshape(conv2, [-1, 7 * 7 * 64])

fc1_logits = fc_layer(flattened, 7 * 7 * 64, 1024, name="fc1")
fc1 = tf.nn.relu(fc1_logits)
logits = fc_layer(fc1, 1024, 10, name="fc2")

## 損失関数と最適化手法

In [9]:
# 損失関数にはクロスエントロピーを使用
with tf.name_scope("xent"):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    tf.summary.scalar("xent", cross_entropy)    

In [10]:
# 最適化手法にはAdamを使用
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [11]:
# 正答率を計算
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)    

## 学習

In [12]:
sess = tf.InteractiveSession()

In [13]:
# 変数の初期化
sess.run(tf.global_variables_initializer())

In [14]:
# TensorBoard用の準備
writer = tf.summary.FileWriter(LOGDIR)  # FileWriterとイベントファイルを作成

In [15]:
# 計算グラフの可視化
writer.add_graph(sess.graph)  # 計算グラフ（Graph）をイベントファイルに書き出す

In [16]:
# summary opからの情報を集める処理
merged_summary = tf.summary.merge_all()

In [17]:
# 250ステップ学習
for i in range(251):
    
    batch = mnist.train.next_batch(100)
    
    if i % 5 == 0:
        [train_accuracy, s] = sess.run([accuracy, merged_summary], feed_dict={x: batch[0], y: batch[1]})
        writer.add_summary(s, global_step=i)
        print("step %d, training accuracy %g" % (i, train_accuracy))
        
    if i % 50 == 0:
        [test_accuracy] = sess.run([accuracy], feed_dict={x: mnist.test.images[:1024], y: mnist.test.labels[:1024]})
        print("---- step %d, test accuracy %g" % (i, test_accuracy))
        
    # 学習
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})
    
writer.close()

step 0, training accuracy 0.07
---- step 0, test accuracy 0.0849609
step 5, training accuracy 0.15
step 10, training accuracy 0.1
step 15, training accuracy 0.08
step 20, training accuracy 0.07
step 25, training accuracy 0.08
step 30, training accuracy 0.09
step 35, training accuracy 0.16
step 40, training accuracy 0.18
step 45, training accuracy 0.11
step 50, training accuracy 0.1
---- step 50, test accuracy 0.126953
step 55, training accuracy 0.12
step 60, training accuracy 0.09
step 65, training accuracy 0.08
step 70, training accuracy 0.08
step 75, training accuracy 0.1
step 80, training accuracy 0.14
step 85, training accuracy 0.07
step 90, training accuracy 0.09
step 95, training accuracy 0.13
step 100, training accuracy 0.05
---- step 100, test accuracy 0.126953
step 105, training accuracy 0.1
step 110, training accuracy 0.13
step 115, training accuracy 0.14
step 120, training accuracy 0.17
step 125, training accuracy 0.13
step 130, training accuracy 0.11
step 135, training accu

## TensorBoardの呼び出し

tensorboard --logdir /tmp/mnist_demo/3